In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
from tensorflow import keras
from sklearn.neighbors import kneighbors_graph, NearestNeighbors
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Model, Sequential, save_model, load_model
from tensorflow.config import list_logical_devices
from tensorflow.keras.callbacks import TensorBoard

In [77]:
# df_full.to_csv('model_ready_data_no_dupes.csv', index=False)

In [3]:
df_full = pd.read_csv(r'C:\Users\Logan\Desktop\model_ready_data_no_dupes.csv')

In [4]:
df = df_full.select_dtypes('number')

In [5]:
data = df.to_numpy()

In [6]:
data.shape

(524211, 15)

In [7]:
df_full

,name,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
0,carve,0.022576,0,"[""'uli'""]",1922-02-22,0.645,0.4450,0.000000,0.713748,1,0.4510,0.674,0.744000,0.1510,0.1270,0.425564,0.6,0.06
1,capítulo 2.16 - banquero anarquista,0.017470,0,"[""'fernando pessoa'""]",1922-06-01,0.695,0.2630,0.000000,0.579173,1,0.9570,0.797,0.000000,0.1480,0.6550,0.414029,0.2,0.00
2,vivo para quererte - remasterizado,0.032313,0,"[""'ignacio corsini'""]",1922-03-21,0.434,0.1770,0.090909,0.593796,1,0.0512,0.994,0.021800,0.2120,0.4570,0.529335,1.0,0.00
3,el prisionero - remasterizado,0.031471,0,"[""'ignacio corsini'""]",1922-03-21,0.321,0.0946,0.636364,0.490073,1,0.0504,0.995,0.918000,0.1040,0.3970,0.689907,0.6,0.00
4,lady of the evening,0.029012,0,"[""'dick haymes'""]",1922,0.402,0.1580,0.272727,0.659263,0,0.0390,0.989,0.130000,0.3110,0.1960,0.418945,0.8,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524206,云与海,0.045945,0,"[""'阿yueyue'""]",2020-09-26,0.560,0.5180,0.000000,0.803491,0,0.0292,0.785,0.000000,0.0648,0.2110,0.535333,0.8,0.50
524207,blind,0.027270,0,"[""'role model'""]",2020-10-21,0.765,0.6630,0.000000,0.837876,1,0.0652,0.141,0.000297,0.0924,0.6860,0.609183,0.8,0.72
524208,what they'll say about us,0.033374,0,"[""'finneas'""]",2020-09-02,0.535,0.3140,0.636364,0.721626,0,0.0408,0.895,0.000150,0.0874,0.0663,0.588905,0.8,0.70
524209,a day at a time,0.025262,0,"[""'gentle bones'"", ""'clara benin'""]",2021-03-05,0.696,0.6150,0.909091,0.822748,1,0.0345,0.206,0.000003,0.3050,0.4380,0.365406,0.8,0.58


In [86]:
class AutoEncoder(Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Sequential([
            Dense(64, input_shape=(data.shape[1],), activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(8, activation='gelu')])
        self.decoder = Sequential([
            Dense(16, activation='gelu'),
            Dense(32, activation='relu'),
            Dense(64, activation='relu'),
            Dense(15, activation='relu')])


    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [212]:
ae = AutoEncoder()
ae.compile(optimizer='adam', loss='mae')

In [213]:
ae.fit(data, data, shuffle=True, epochs=10, workers=10, use_multiprocessing=True, validation_split=.1)

Epoch 1/10
14744/14744 [==============================] - 13s 853us/step - loss: 0.1855 - val_loss: 0.1753
Epoch 2/10
14744/14744 [==============================] - 13s 856us/step - loss: 0.1559 - val_loss: 0.1564
Epoch 3/10
14744/14744 [==============================] - 12s 839us/step - loss: 0.1524 - val_loss: 0.1531
Epoch 4/10
14744/14744 [==============================] - 12s 836us/step - loss: 0.1509 - val_loss: 0.1536
Epoch 5/10
14744/14744 [==============================] - 12s 839us/step - loss: 0.1506 - val_loss: 0.1528
Epoch 6/10
14744/14744 [==============================] - 12s 836us/step - loss: 0.1504 - val_loss: 0.1520
Epoch 7/10
14744/14744 [==============================] - 13s 856us/step - loss: 0.1503 - val_loss: 0.1529
Epoch 8/10
14744/14744 [==============================] - 12s 842us/step - loss: 0.1501 - val_loss: 0.1527
Epoch 9/10
14744/14744 [==============================] - 12s 830us/step - loss: 0.1500 - val_loss: 0.1521
Epoch 10/10
14744/14744 [============

In [214]:
encoded_data = ae.encoder(data)

In [215]:
encoded_data

<tf.Tensor: shape=(524211, 8), dtype=float32, numpy=
array([[ 1.4638278 ,  0.43772218,  0.45585892, ..., -0.1414433 ,
        -0.02446401,  1.1333817 ],
       [ 1.0166061 ,  0.47225404,  0.28378925, ..., -0.0944583 ,
        -0.02418888,  1.490889  ],
       [ 1.0410358 ,  0.2693591 ,  0.96988726, ..., -0.08931696,
        -0.03358055,  1.4530045 ],
       ...,
       [ 1.863161  ,  0.8951466 ,  0.8856626 , ..., -0.11445237,
        -0.09538402,  1.528984  ],
       [ 1.5459175 ,  1.489423  ,  0.7626769 , ..., -0.110218  ,
        -0.1341368 ,  1.081506  ],
       [ 0.84015423,  1.5982221 ,  0.933198  , ..., -0.10198129,
        -0.09008599,  1.0321317 ]], dtype=float32)>

In [216]:
knn = NearestNeighbors(n_neighbors=5, radius=1, n_jobs=-1)
knn.fit(encoded_data)

NearestNeighbors(n_jobs=-1, radius=1)

In [93]:
model_2 = AutoEncoder()
loss = tf.keras.losses.MeanAbsoluteError()
model_2.compile(optimizer='nadam', loss=loss)

In [94]:
model_2.fit(data, data, epochs=10, shuffle=True, validation_split=.1)

Epoch 1/10
14744/14744 [==============================] - 14s 837us/step - loss: 0.1466 - val_loss: 0.1339
Epoch 2/10
14744/14744 [==============================] - 12s 820us/step - loss: 0.1347 - val_loss: 0.1262
Epoch 3/10
14744/14744 [==============================] - 12s 824us/step - loss: 0.1037 - val_loss: 0.0880
Epoch 4/10
14744/14744 [==============================] - 12s 825us/step - loss: 0.0902 - val_loss: 0.0853
Epoch 5/10
14744/14744 [==============================] - 12s 835us/step - loss: 0.0832 - val_loss: 0.0727
Epoch 6/10
14744/14744 [==============================] - 12s 835us/step - loss: 0.0753 - val_loss: 0.0731
Epoch 7/10
14744/14744 [==============================] - 12s 817us/step - loss: 0.0751 - val_loss: 0.0726
Epoch 8/10
14744/14744 [==============================] - 12s 811us/step - loss: 0.0750 - val_loss: 0.0714
Epoch 9/10
14744/14744 [==============================] - 12s 819us/step - loss: 0.0748 - val_loss: 0.0713
Epoch 10/10
14744/14744 [============

In [95]:
encoded_data_2 = model_2.encoder(data)

In [96]:
knn_2 = NearestNeighbors(n_neighbors=5, n_jobs=-1)
knn_2.fit(encoded_data_2)

NearestNeighbors(n_jobs=-1)

In [97]:
class AutoEncoder2(Model):
    def __init__(self, alpha):
        super(AutoEncoder2, self).__init__()
        self.encoder = Sequential([
            Dense(32, input_shape=data.shape[1:]),
            LeakyReLU(alpha),
            Dense(16),
            LeakyReLU(alpha),
            Dense(5),
            LeakyReLU(alpha)])
        self.decoder = Sequential([
            Dense(16),
            LeakyReLU(alpha),
            Dense(32),
            LeakyReLU(alpha),
            Dense(15),
            LeakyReLU(alpha)])
    

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [217]:
ae2 = AutoEncoder2(.1)
ae2.compile(optimizer='adam', loss='mae')

In [218]:
ae2.fit(data, data, epochs=10, validation_split=.1, workers=10, use_multiprocessing=True, shuffle=True)

Epoch 1/10
14744/14744 [==============================] - 11s 716us/step - loss: 0.0616 - val_loss: 0.0483
Epoch 2/10
14744/14744 [==============================] - 10s 696us/step - loss: 0.0529 - val_loss: 0.0465
Epoch 3/10
14744/14744 [==============================] - 10s 700us/step - loss: 0.0504 - val_loss: 0.0459
Epoch 4/10
14744/14744 [==============================] - 10s 689us/step - loss: 0.0490 - val_loss: 0.0445
Epoch 5/10
14744/14744 [==============================] - 10s 686us/step - loss: 0.0486 - val_loss: 0.0452
Epoch 6/10
14744/14744 [==============================] - 10s 703us/step - loss: 0.0485 - val_loss: 0.0441
Epoch 7/10
14744/14744 [==============================] - 10s 680us/step - loss: 0.0484 - val_loss: 0.0442
Epoch 8/10
14744/14744 [==============================] - 10s 679us/step - loss: 0.0483 - val_loss: 0.0444
Epoch 9/10
14744/14744 [==============================] - 10s 675us/step - loss: 0.0481 - val_loss: 0.0441
Epoch 10/10
14744/14744 [============

In [219]:
encoded_data_3 = ae2.encoder(data)

In [220]:
knn_3 = NearestNeighbors(n_neighbors=5, n_jobs=-1)
knn_3.fit(encoded_data_3)

NearestNeighbors(n_jobs=-1)

In [252]:
ae4 = AutoEncoder2(.3)
ae4.compile(optimizer='rmsprop', loss='mae')

In [253]:
ae4.fit(data, data, epochs=10, validation_split=.1, workers=10, use_multiprocessing=True, shuffle=True)

Epoch 1/10
14744/14744 [==============================] - 10s 642us/step - loss: 0.0648 - val_loss: 0.0542
Epoch 2/10
14744/14744 [==============================] - 9s 637us/step - loss: 0.0579 - val_loss: 0.0526
Epoch 3/10
14744/14744 [==============================] - 10s 654us/step - loss: 0.0510 - val_loss: 0.0452
Epoch 4/10
14744/14744 [==============================] - 10s 649us/step - loss: 0.0485 - val_loss: 0.0428
Epoch 5/10
14744/14744 [==============================] - 9s 642us/step - loss: 0.0477 - val_loss: 0.0431
Epoch 6/10
14744/14744 [==============================] - 9s 644us/step - loss: 0.0467 - val_loss: 0.0418
Epoch 7/10
14744/14744 [==============================] - 9s 631us/step - loss: 0.0455 - val_loss: 0.0399
Epoch 8/10
14744/14744 [==============================] - 9s 636us/step - loss: 0.0448 - val_loss: 0.0420
Epoch 9/10
14744/14744 [==============================] - 9s 637us/step - loss: 0.0443 - val_loss: 0.0414
Epoch 10/10
14744/14744 [==================

In [257]:
ae4.save('ae4')

INFO:tensorflow:Assets written to: ae4\assets


In [16]:
ae4 = load_model('ae4')

In [17]:
encoded_data_4 = ae4.encoder(data)
knn4 = NearestNeighbors(n_jobs=-1)
knn4.fit(encoded_data_4)

NearestNeighbors(n_jobs=-1)

In [19]:
joblib.dump(encoded_data_4, 'encoded_data.joblib')

['encoded_data.joblib']

In [255]:
query = 72837

_, ind = knn.kneighbors([encoded_data[query]])
_, ind2 = knn_2.kneighbors([encoded_data_2[query]])
_, ind3 = knn_3.kneighbors([encoded_data_3[query]])
_, ind4 = knn4.kneighbors([encoded_data_4[query]])

In [243]:
df_full.iloc[ind[0]]

,name,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
72837,duality,0.044939,0,"[""'slipknot'""]",2004,0.354,0.982,0.363636,0.871482,0,0.1680,0.000237,0.000294,0.2040,0.194,0.583535,0.8,0.78
76442,use somebody,0.041052,0,"[""'kings of leon'""]",2008-09-23,0.276,0.715,0.000000,0.835842,1,0.0432,0.005520,0.000417,0.2010,0.173,0.556163,0.8,0.77
74796,never too late,0.037240,0,"[""'three days grace'""]",2006-06-25,0.433,0.778,0.181818,0.846182,0,0.0371,0.008380,0.000000,0.1630,0.217,0.608261,0.8,0.70
72828,mr. brightside,0.039666,0,"[""'the killers'""]",2004,0.352,0.911,0.090909,0.837769,1,0.0747,0.001210,0.000000,0.0995,0.236,0.600830,0.8,0.82
165065,crazy girl,0.035705,0,"[""'eli young band'""]",2011-01-01,0.366,0.651,0.363636,0.859000,1,0.0310,0.000415,0.005260,0.2710,0.257,0.614264,0.6,0.69


In [244]:
df_full.iloc[ind2[0]]

,name,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
72837,duality,0.044939,0,"[""'slipknot'""]",2004,0.354,0.982,0.363636,0.871482,0,0.1680,0.000237,0.000294,0.2040,0.194,0.583535,0.8,0.78
50622,ace of spades,0.029695,0,"[""'motörhead'""]",1980-11-08,0.329,0.974,0.272727,0.783621,0,0.1350,0.000852,0.000118,0.0904,0.234,0.571724,0.8,0.75
71173,times like these,0.047242,0,"[""'foo fighters'""]",2002-10-22,0.376,0.908,0.363636,0.859750,0,0.0879,0.000014,0.000014,0.2410,0.266,0.586819,0.8,0.68
74801,knights of cydonia,0.065148,0,"[""'muse'""]",2006-06-19,0.366,0.963,1.000000,0.836683,0,0.1420,0.000273,0.012200,0.1150,0.211,0.556512,0.8,0.69
73759,best of you,0.045475,0,"[""'foo fighters'""]",2005-06-14,0.366,0.940,0.090909,0.839467,0,0.0696,0.000769,0.000094,0.1880,0.369,0.528442,0.8,0.76


In [245]:
df_full.iloc[ind3[0]]

,name,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
72837,duality,0.044939,0,"[""'slipknot'""]",2004,0.354,0.982,0.363636,0.871482,0,0.1680,0.000237,0.000294,0.2040,0.194,0.583535,0.8,0.78
164331,inside out,0.040267,1,"[""'five finger death punch'""]",2019-12-02,0.476,0.991,0.363636,0.874098,0,0.1500,0.001910,0.000000,0.1210,0.202,0.336864,0.8,0.62
79753,kyoto (feat. sirah),0.035719,1,"[""'skrillex'"", ""'sirah'""]",2011-12-27,0.605,0.920,0.363636,0.882617,0,0.0713,0.001200,0.000596,0.4830,0.219,0.710177,0.8,0.61
352434,final round,0.029492,0,"[""'free flow flava'""]",2019-10-04,0.342,0.939,0.363636,0.798168,0,0.1160,0.002480,0.857000,0.1790,0.243,0.696795,0.8,0.56
54565,creeping death - creeping death ep version / r...,0.070395,0,"[""'metallica'""]",1984-07-26,0.251,0.978,0.363636,0.833716,0,0.1660,0.000254,0.315000,0.0798,0.170,0.409212,0.8,0.57


In [256]:
df_full.iloc[ind4[0]]

,name,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
72837,duality,0.044939,0,"[""'slipknot'""]",2004,0.354,0.982,0.363636,0.871482,0,0.1680,0.000237,0.000294,0.2040,0.194,0.583535,0.8,0.78
71911,bring me to life,0.041965,0,"[""'evanescence'""]",2003-03-04,0.331,0.943,0.363636,0.869004,0,0.0698,0.007210,0.000002,0.2420,0.296,0.384007,0.8,0.81
77786,eyeless,0.042048,1,"[""'slipknot'""]",2009-09-09,0.293,0.997,0.363636,0.851016,0,0.2170,0.000463,0.000039,0.4150,0.130,0.407288,0.8,0.61
50622,ace of spades,0.029695,0,"[""'motörhead'""]",1980-11-08,0.329,0.974,0.272727,0.783621,0,0.1350,0.000852,0.000118,0.0904,0.234,0.571724,0.8,0.75
73246,"give 'em hell, kid",0.024699,0,"[""'my chemical romance'""]",2004-06-08,0.252,0.993,0.363636,0.870671,0,0.1600,0.023400,0.000000,0.2280,0.118,0.745889,0.8,0.62


In [204]:
df_full[df_full['artists'].str.contains('slipkn') == True].head(50)

,name,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,popularity
70349,people = shit,0.038355,1,"[""'slipknot'""]",2001,0.462,0.996,1.000000,0.872293,0,0.2310,0.000058,0.003770,0.1390,0.1450,0.495460,0.8,0.66
70486,the heretic anthem,0.045186,1,"[""'slipknot'""]",2001,0.451,0.993,0.090909,0.854396,1,0.1850,0.000562,0.004790,0.1520,0.1400,0.409232,0.8,0.63
70494,left behind,0.042956,0,"[""'slipknot'""]",2001,0.229,0.994,0.545455,0.865287,1,0.2200,0.000070,0.005480,0.3180,0.1150,0.635694,0.8,0.63
70657,disasterpiece,0.054847,1,"[""'slipknot'""]",2001,0.395,0.988,1.000000,0.855620,0,0.2460,0.000128,0.000244,0.1340,0.1890,0.437063,0.8,0.60
70700,my plague,0.039190,1,"[""'slipknot'""]",2001,0.279,0.993,1.000000,0.876927,0,0.2090,0.000278,0.000394,0.1970,0.1070,0.359415,0.8,0.60
70951,everything ends,0.045205,1,"[""'slipknot'""]",2001,0.467,0.992,1.000000,0.868545,0,0.1750,0.000078,0.001750,0.4100,0.3170,0.467605,0.8,0.57
72837,duality,0.044939,0,"[""'slipknot'""]",2004,0.354,0.982,0.363636,0.871482,0,0.1680,0.000237,0.000294,0.2040,0.1940,0.583535,0.8,0.78
72846,before i forget,0.049562,0,"[""'slipknot'""]",2004,0.291,0.974,0.545455,0.871925,0,0.1710,0.017700,0.000005,0.8860,0.3480,0.546812,0.8,0.75
72985,"vermilion, pt. 2",0.039906,0,"[""'slipknot'""]",2004,0.589,0.442,0.363636,0.787491,1,0.0291,0.334000,0.000424,0.0987,0.0904,0.497960,0.6,0.67
73263,vermilion,0.056334,0,"[""'slipknot'""]",2004,0.219,0.989,1.000000,0.872461,0,0.1600,0.004210,0.015600,0.3750,0.1490,0.682151,0.6,0.62


In [8]:
model = load_model('ae4')